# Continuous Portfolio Optimization

#### Device: Dirac-3


Continuous optimization helps in selecting and determining how much to invest in each stock. Unlike binary optimization, where you decide whether to include a stock or not (1 or 0), continuous optimization allows for more precise investment decisions by assigning continuous values to represent the proportion of your investment in each stock. This method helps create a well-diversified portfolio with varying levels of investment in different stocks to better manage risk and achieve desired returns.


![weighted stocks](img/continuous/01.svg)

## Constructing our Objective Function

In our tutorial, we have historical datasets containing stock prices over a period of time. Our goal is to use these datasets to calculate important metrics such as [the rate of return](https://en.wikipedia.org/wiki/Rate_of_return), [expected return](https://en.wikipedia.org/wiki/Expected_return), [variance](https://en.wikipedia.org/wiki/Variance), and [covariance](https://en.wikipedia.org/wiki/Covariance). By calculating these metrics for each stock in our portfolio, we gain insights into their performance and risk characteristics. Expected return helps us gauge potential profitability, variance quantifies the risk associated with individual stocks, and covariance indicates how stocks move relative to each other. These calculations are essential for making informed investment decisions aimed at optimizing returns while managing risk effectively.


We define the following variables as:

**1. $i$:** Index of stock

**2. K:** Total number of available stocks in given portfolio.

**3. R:** Daily returns or the Returns per Day of the portfolio

**4. E(R):** Expected Return of the portfolio

**5. Var(R):** Variance of portfolio's returns

**6. ξ:** A parameter that balances the importance of maximizing returns versus minimizing risk.

**7. $R_b$:** Base interest rate.e; it is introduced so that the
two added terms in the objective function are on an equal footing.

**8. $w_i$:**  Weight of
stock i in the portfolio; $i ∈ {1, 2, ..., K}$


Now, we can define our **Objective Function**, i.e, the function that encapsulates the goal of constructing a portfolio that balances maximizing expected returns and minimizing risk.   as follows:


$$\text{min}_{\{w_i\}_i𝞊\{1,2,..k\}} [  -E(R) \cdot R_b + ξ\cdot \text{Var}(R)$$



We want our weights to sum to $100%$, therefore:

$$ ∑_{i=1}^k w_i = 100$$



We also want the portfolio to be diverse, so we apply an upper limit on all the weights:

$$ w_i \leq W_{\text{max}}$$

In our example we set our  upper limit on all the weights $W_{\text{max}}$ equal to $80$.

## Hamiltonian Construction

We are going to get into more rigorous math in this section. So you can feel free to skip to the next section to understand the code flow and observe the results!


The portfolio daily return over a time period $m$ can be written as a linear combination of daily returns of its constituent stocks over the same time period. We have,

$$ R^{(m)}(t) = ∑_{i = 1}^K w_i r_i ^{(m)}(t) $$

where, $r_i ^{(m)}(t)$ is the daily return of stock $i$ at time $t$ during time period $m$.

The expectation of portfolio daily return over time period $m$ can thus be expanded as:
 $$ E(R^{(m)}) = ∑_{i = 1}^K w_i E(r_i ^{(m)})$$

and the variance of portfolio daily returns over time period $m$ is expanded as,

$$ \text{Var}(R^{(m)}) = ∑_{i = 1}^K ∑_{j = 1}^K w_i w_j \text{Covar}(r_i ^{(m)},r_j ^{(m)}) $$

where Covar is the covariant function.



To impose the inequality constraints in $ w_i \leq W_{\text{max}}$, we can introduce $K$ auxiliary variables
$w_{K+1}, w_{K+2}, ..., w_{2K}$, and impose $K$ equality constraints as,

$$w_i + w_{i+K} = W_{\text{max}} $$


Substituting Equations for daily expectation of portfolio daily return over time period $m$ and variance of portfolio daily returns over time period $m$ into Equation 2, the objective function, that we now call $f^{(m)},$ becomes:

$$ f^{(m)}(w) = ∑_{i,j = 1} ^K w_i A_{ij}^{(m)}w_j + \sum_{i=1}^K w_ib_i^{(m)} + α(∑_{i=1}^K w_i - 100 )^2 + β(∑_{i=1}^K (w_i + w_{i+K} - W_{\text{max}})^2 $$

where,

$$ A^{(m)}_{ij} = ξ\cdot \text{Covar}(r^{(m)}_i, r^{(m)}_j) $$

$$b^{(m)}_i = −R_B\cdot E(r^{(m)}_i)$$


To avoid an over-fit on the portfolio data, we can minimize the average of the cost function over $M$
overlapping time periods, that is $m ∈ {1, 2, ..., M}.$ The problem becomes,

$$ f(w) =  \frac{1}{M} ∑_{m=1}^M f(w)$$


Adding the constraints to the objective function, the optimization
problem reduces to,

$$ \text{min}_{w_i} ∑_{i=1}^{2K} ∑_{j=1}^{2K} J_{ij} w_iw_j + ∑_{i=1}^{2k}h_iw_i $$

where the two body coupling coefficients J_{ij} are defined as

$J_{ij} = \left\{
  \begin{array}{cl}
    \frac{1}{M} \sum_{m=1}^M A_{ij}^{(m)} + \alpha + \beta \delta_{ij} & \quad \text{if } i,j \leq K \\
    \beta & \quad \text{if } i \leq K, j = i + K \\
    \beta & \quad \text{if } i = j + K, j \leq K \\
    \beta \delta_{ij} & \quad \text{if } i > K, j > K \\
    0 & \quad \text{otherwise}
  \end{array}
\right.$


and the linear coefficients $h_i$ are defined as,

$h_{i} = \left\{
  \begin{array}{cl}
    b_i - 200\alpha - 2\beta W_{\text{max}}& \quad \text{if } i \leq K \\
    -2\beta W_{\text{max}} & \quad \text{if } i \geq K
  \end{array}
\right.$

The Hamiltonian equation becomes:


$H = [ h \mid \frac{1}{2} (J + J^T)]$

where:

- $( 0.5 \cdot (J + J^T) )$ represents the symmetrized form of $J$
- The notation $h \mid \frac{1}{2} (J + J^T)$ denotes the [horizontal concatenation](https://www.mathworks.com/help/matlab/ref/double.horzcat.html) of $h$ and $\frac{1}{2} (J + J^T)$.

## Advantages of Using Dirac-3 for Portfolio Optimization
### Challenges with Traditional Quantum Computing Methods

In portfolio optimization, we aim to minimize the cost function:
$$ \text{min}_{w_i} ∑_{i=1}^{2K} ∑_{j=1}^{2K} J_{ij} w_iw_j + ∑_{i=1}^{2k}h_iw_i $$

This involves continuous variables $ w_i $. To solve this on a quantum computer, we typically need to convert the problem into a format called Quadratic Unconstrained Binary Optimization (QUBO) [1, 2]. This requires an additional embedding step.

### Increased Problem Size and Accuracy-Complexity Trade-off

Converting continuous variables into binary ones makes the problem much larger. Achieving higher accuracy requires more binary variables, which can exceed the hardware's capabilities, increasing both the complexity and the required dynamic range.

### Limited Connectivity Issues

Quantum computers often lack global connectivity, meaning not all qubits can interact directly. This necessitates mapping the problem onto the hardware's specific qubit arrangement, further increasing the problem size and the time needed to find a solution [3, 4].

### Advantages of Dirac-3

Dirac-3 quantum optimization machine offers a significant advantage over traditional methods. It can work directly with discrete variables encoded in a high-dimensional time-energy mode, allowing it to handle the portfolio optimization problem without needing the conversion step. This eliminates the issues related to limited connectivity and simplifies the optimization process.

# Understanding the Code with an Example

Before we begin, you'll need your unique token to access the QCi Client API and connect to the Dirac device. If you don't have a token yet, you can sign up for our [Free Trial Cloud Access](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/quick-start-on-cloud)). Let's get started!

You can download the data sets and get access to the full code base [here](https://git.qci-dev.com/cvadlamani/basic-catalyst-tutorial/-/tree/Chitra/integer_portfolio_optimization?ref_type=heads)

Imagine you have  $5$  stocks in your portfolio and you want to find the optimized portfolio weights

Here are 5 stocks present in our portfolio, let us see how we can optimize them using our Run() method:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stock</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>AAPL</td>
    </tr>
    <tr>
      <th>1</th>
      <td>AMZN</td>
    </tr>
    <tr>
      <th>2</th>
      <td>GOOG</td>
    </tr>
    <tr>
      <th>3</th>
      <td>MSFT</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NVDA</td>
    </tr>
  </tbody>
</table>


## Run () method

The *run function* is the main function of the portfolio optimization pipeine that performs a series of operations to analyze stock data and optimize a stock portfolio based on historical returns.

It does three main things: first, it retrieves the stock data. Then it preprocesses the data and  constructs a Hamiltonian matrix to balance returns and risks. Finally, it optimizes the stock portfolio and the function ultimately saves the stock allocation data to a specified CSV file and displays the optimized portfolio

![flow diagram](img/continuous/02.svg)

## Step 1

Initialize and Prepare Dates:
* Calculates the in-sample date range based on the current date for historical data analysis.

In [ ]:
current_date  = pd.to_datetime(current_date )
in_sample_start_date  = current_date  - datetime.timedelta(days=in_sample_days)
in_sample_end_date  = current_date - datetime.timedelta(days=1)

Retrieve Stock Data:
* Obtains the list of stock constituents as of the current_date.
* Fetches historical stock returns for the in-sample period.

In [ ]:
stocks = get_constituents(current_date)
in_sample_returns_df  = get_stock_returns(stocks, in_sample_start_date, in_sample_end_date)

## Step 2
Preprocess Data:
* Sorts the stock return data by date.
* Fills any missing values in the stock return data.

In [ ]:
in_sample_returns_df  = in_sample_returns_df.sort_values("Date")
in_sample_returns_df  = in_sample_returns_df.fillna(method="ffill").fillna(0)

## Step 3
Calculates Elements of Hamiltonian
* Computes the elements $J$ and $h$, and a sum constraint using the in-sample stock returns.


In [ ]:
J, h, sum_constraint = get_hamiltonian(
    in_sample_returns_df,
    stocks,
    in_sample_start_date ,
    in_sample_end_date ,
)

np.save("J_%s.npy" % current_date.strftime("%Y-%m-%d"), J)
np.save("h_%s.npy" % current_date.strftime("%Y-%m-%d"), h)



## Step 4
* Optimize Portfolio:
  * Creates Hamiltonian using Hamiltonian data.
  * Runs a portfolio optimization algorithm using the Hamiltonian.
  * Saves the optimization results and stock list to files.


In [ ]:
  optimized_weights , stock_allocations = optimize_portfolio(J, h, sum_constraint, stocks)

The result JSON file looks like this:

In [ ]:
{
  "job_info": {
    "job_id": "6679944a450099160ba0bf88",
    "job_submission": {
      "job_tags": ["port_opot_nasdaq100_d3"],
      "problem_config": {
        "normalized_qudit_hamiltonian_optimization": {
          "hamiltonian_file_id": "6679944a98263204a365f329"
        }
      },
      "device_config": {
        "dirac-3": {
          "num_samples": 1,
          "relaxation_schedule": 1,
          "sum_constraint": 40
        }
      }
    },
    "job_status": {
      "submitted_at_rfc3339nano": "2024-06-24T15:44:10.523Z",
      "queued_at_rfc3339nano": "2024-06-24T15:44:10.524Z",
      "running_at_rfc3339nano": "2024-06-24T15:44:11.514Z",
      "completed_at_rfc3339nano": "2024-06-24T15:44:13.339Z"
    },
    "job_result": {
      "file_id": "6679944d98263204a365f32b",
      "device_usage_s": 2
    }
  },
  "status": "COMPLETED",
  "results": {
    "counts": [1],
    "energies": [-32319.5527344],
    "solutions": [
      [8.0512924, 7.9148078, 7.9524417, 7.9162955, 8.1651621, 0, 0, 0, 0, 0]
    ]
  }
}

Where the following fields are:

Certainly! Here's a breakdown of each field in the JSON:

- **job_info**: Contains details about the job.
  - **job_id**: Unique identifier for the job (`6679944a450099160ba0bf88`).
  - **job_submission**: Information about the job submission.
    - **job_tags**: Tags associated with the job (`port_opot_nasdaq100_d3`).
    - **problem_config**: Configuration of the problem to be solved.
      - **normalized_qudit_hamiltonian_optimization**: Specifies the problem type as normalized qudit Hamiltonian optimization.
        - **hamiltonian_file_id**: ID of the file containing the Hamiltonian information (`6679944a98263204a365f329`).
    - **device_config**: Configuration of the device used for the job.
      - **dirac-3**: Specifies the device used (`dirac-3`).
        - **num_samples**: Number of samples taken (value: 1).
        - **relaxation_schedule**: Relaxation schedule parameter (value: 1).
        - **sum_constraint**: Sum constraint parameter (value: 40).
  - **job_status**: Status updates of the job.
    - **submitted_at_rfc3339nano**: Time when the job was submitted (`2024-06-24T15:44:10.523Z`).
    - **queued_at_rfc3339nano**: Time when the job was queued (`2024-06-24T15:44:10.524Z`).
    - **running_at_rfc3339nano**: Time when the job started running (`2024-06-24T15:44:11.514Z`).
    - **completed_at_rfc3339nano**: Time when the job completed (`2024-06-24T15:44:13.339Z`).
  - **job_result**: Results of the job.
    - **file_id**: ID of the file containing the job result (`6679944d98263204a365f32b`).
    - **device_usage_s**: Time the device was used, in seconds (value: 2).

- **status**: The current status of the job (`COMPLETED`).

- **results**: The outcomes of the job.
  - **counts**: Number of samples taken for each solution (value: [1]).
  - **energies**: Energy values associated with the solutions (value: [-32319.5527344]).
  - **solutions**: The actual solutions found, represented as a list of values (value: [[8.0512924, 7.9148078, 7.9524417, 7.9162955, 8.1651621, 0, 0, 0, 0, 0]]).




The solution weights are then adjusted to form a balanced portfolio and we obtain:

```python
 Optimal_Portfolio_with_allocations = {
    "AAPL": 0.2012823125160289,
    "NVDA": 0.19787019747337747,
    "AMZN": 0.19881104498513807,
    "MSFT": 0.19790738997384238,
    "GOOG": 0.20412905505161316
  }

Where The optimal allocations for the portfolio are as follows:
  - **AAPL**: 20.13%
  - **NVDA**: 19.79%
  - **AMZN**: 19.88%
  - **MSFT**: 19.79%
  - **GOOG**: 20.41%

## Step 5
Generate and Save Allocation Data:
* Creates a DataFrame with stock allocations.
* Saves the allocation data to a specified CSV file, appending if the file already exists.

In [ ]:
np.save("qci_sol_%s.npy" % current_date.strftime("%Y-%m-%d"), sol)
np.save("stock_list_%s.npy" % current_date.strftime("%Y-%m-%d"), stocks)    
weight_df = pd.DataFrame(
    {
        "Stock": [item for item in stock_allocations.keys()],
        "Allocation": [
            stock_allocations[item] for item in stock_allocations.keys()
        ],
    }
)
weight_df["Date"] = current_date
weight_df = weight_df[weight_df["Allocation"] > 0]
weight_df["Stock_Count"] = weight_df.shape[0]
if os.path.exists(ALLOC_OUT_FILE):
    weight_df.to_csv(
        ALLOC_OUT_FILE,
        index=False,
        mode="a",
        header=False,
    )
else:
    weight_df.to_csv(
        ALLOC_OUT_FILE,
        index=False,
    )

Display optimized portfolio

In [ ]:
display(HTML(weight_df[["Stock","Allocation"]].tail(5).to_html()))


## Optimial Portfilio with Allocated weights:

When we run our code, we will see that the optimal portfolio has the following stock allocations:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stock</th>
      <th>Allocation</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NVDA</td>
      <td>0.196433</td>
    </tr>
    <tr>
      <th>1</th>
      <td>AMZN</td>
      <td>0.194561</td>
    </tr>
    <tr>
      <th>2</th>
      <td>MSFT</td>
      <td>0.211609</td>
    </tr>
    <tr>
      <th>3</th>
      <td>AAPL</td>
      <td>0.195888</td>
    </tr>
    <tr>
      <th>4</th>
      <td>GOOG</td>
      <td>0.201509</td>
    </tr>
  </tbody>
</table>

![weighted chart](img/continuous/03.svg)

## Conclusion

In conclusion, we observe that Dirac effectively finds the solution and successfully solves the portfolio optimization problem. For a more challenging and comprehensive use case, refer to our example tutorial on optimizing a 100-asset portfolio over a 21-year period using the Nasdaq-100 as a benchmark with the Dirac-3 machine. Click [here](https://git.qci-dev.com/qci-dev/quantum-solutions/-/blob/main/qc/portfolio_optimization_dirac_3/build_unequal_nasdaq100_dirac3.py?ref_type=heads) to learn more.


## Citations
[1] R. Dridi and H. Alghassi. “Prime factorization using quantum annealing and computational
algebraic geometry”. In: Sci Rep 7 (2017).[ DOI: https://doi.org/10.1038/srep43048.](https://www.nature.com/articles/srep43048)

[2] Akshay Ajagekar, Kumail Al Hamoud, and Fengqi You. “Hybrid Classical-Quantum Optimization
Techniques for Solving Mixed-Integer Programming Problems in Production Scheduling”. In:
IEEE Transactions on Quantum Engineering 3 (2022), pp. 1–16.[ DOI: 10.1109/TQE.2022.3187367.](https://ieeexplore.ieee.org/document/9810536)

[3] Yanjun Ji et al. Algorithm-oriented qubit mapping for variational quantum algorithms. 2024. arXiv:
2310.09826 [quant-ph](https://arxiv.org/abs/2310.09826).

[4] Adam Holmes et al. “Impact of qubit connectivity on quantum algorithm performance”. In:
Quantum Sci. Technol (2020)
